# Hackerearth Exhibit Art Challenge
<hr>

<p align="center">
    <img src="https://d2908q01vomqb2.cloudfront.net/cb4e5208b4cd87268b208e49452ed6e89a68e0b8/2021/07/16/HackerEarthFeatureImage.png" width="500" height="600">
</p>

## Problem statement

An art exhibitor is soon to launch an online portal for enthusiasts worldwide to start collecting art with only a click of a button. However, navigating the logistics of selling and distributing art does not seem to be a very straightforward task; such as acquiring art effectively and shipping these artifacts to their respective destinations post-purchase.

## Task

The exhibitor has hired you as a Machine Learning Engineer for this project. You are required to build an advanced model that predicts the cost of shipping paintings, antiques, sculptures, and other collectibles to customers based on the information provided in the dataset.

## Evaluation

<code>score = 100*max(0, 1-metrics.mean_squared_log_error(actual, predicted))</code>

Link : https://www.hackerearth.com/problem/machine-learning/predict-the-cost-to-ship-the-sculptures-12-e7728f5d/

## Environment Setup

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Import for scoring
from sklearn.metrics import mean_squared_log_error

## Gathering the data

In [24]:
# Training dataset
df_train = pd.read_csv('train.csv', parse_dates=["Scheduled Date", "Delivery Date"])

# Testing dataset
df_test = pd.read_csv('test.csv', parse_dates=["Scheduled Date", "Delivery Date"])

# Store test customer id separately
customer_id = df_test['Customer Id']

# Store target separately
target = df_train['Cost']

# Drop target
df_train.drop('Cost', axis=1, inplace=True)

## Helper Functions

In [25]:
def scoring(actual_preds, model_preds):
    '''
    This method will score our model
    '''
    score = 100*max(0, 1-mean_squared_log_error(actual_preds, model_preds))
    return score

def log1p(data):
    '''
    To handle the negative target data before fitting into the model
    '''
    return np.log1p(abs(data))

def expm1(data):
    '''
    To handle the negative target data after making predictions
    '''
    return np.expm1(data)

def submission(model, data, name):
    '''
    Creates csv for submission
    '''
    #data = preprocess(data)
    test_predict = expm1(model.predict(data))
    test_predict_df = pd.DataFrame()
    test_predict_df['Customer Id'] = customer_id
    test_predict_df['Cost'] = test_predict
    test_predict_df.to_csv(name)
    return test_predict_df

## Data Preprocessing

In [33]:
median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mode_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
standardScaler = StandardScaler()

def categorical_numerical(data):
    '''
    Categorical and Numerical columns
    '''
    categorical = []
    numerical = []
    
    for label, content in data.items():
        if pd.api.types.is_string_dtype(content):
            categorical.append(label)
        if pd.api.types.is_numeric_dtype(content):
            numerical.append(label)
            
    return categorical, numerical

def feature_extraction(data):
    '''
    This method will extract the features from the columns
    '''
    # Extract the features from the date columns 'Scheduled Date' and 'Delivery Date'
    data['scheduledYear'] = data['Scheduled Date'].dt.year
    data['scheduledMonth'] = data['Scheduled Date'].dt.month
    data['scheduledDay'] = data['Scheduled Date'].dt.day
    data['deliveryYear'] = data['Delivery Date'].dt.year
    data['deliveryMonth'] = data['Delivery Date'].dt.month
    data['deliveryDay'] = data['Delivery Date'].dt.day
    
    # Extract the features from Customer location
    data['Province'] = data['Customer Location'].map(lambda x:x.split()[-2])
    data['Pincode'] = data['Customer Location'].map(lambda x:x.split()[-1])
    data['Place'] = data['Customer Location'].map(lambda x:x.split(',')[0])
    
    #Convert Pincode into numeric
    data['Pincode'] = data['Pincode'].astype(int, errors='raise')
    
    # Drop the 'Scheduled Date' and 'Delivery Date' columns since features are extracted
    data.drop(['Scheduled Date', 'Delivery Date','Artist Name', 'Customer Id', 'Customer Location'], axis=1, inplace=True)
    
    for label, content in data.items():
        if pd.api.types.is_string_dtype(content):
            data[label] = content.astype("category").cat.as_ordered()
    
    return data

def fill_missing_data(data, categorical_cols, numerical_cols):
    data[numerical_cols] = median_imputer.fit_transform(data[numerical_cols])
    data[categorical_cols] = mode_imputer.fit_transform(data[categorical_cols])
#     for label, content in data.items():
#     # Filling missing numerical data with median
#         if pd.api.types.is_numeric_dtype(content):
#             if pd.isnull(content).sum():
#                 data[label] = content.fillna(content.median())
        
#         # Turning categories into numbers and adding one
#         if not pd.api.types.is_numeric_dtype(content):
#             data[label] = pd.Categorical(content).codes + 1
    return data

def scale_data(data, categorical_cols, numerical_cols):
    for label, content in data.items():
        if not pd.api.types.is_numeric_dtype(content):
            data[label] = pd.Categorical(content).codes + 1
    return data
    
def preprocess(data):
    '''
    This method will preprocess the data.
    '''
    data = feature_extraction(data)
    categorical, numerical = categorical_numerical(data)
    data = fill_missing_data(data, categorical, numerical)
    #data = scale_data(data, categorical, numerical)
    return data

def deep_learning(models, X_train, X_test, y_train, y_test):
    '''
    Fit and Score the deep learning models without performing hyperparameter tuning
    '''
    model_scores = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        model_preds= model.predict(X_test)
        model_scores[name] = scoring(y_test, model_preds)
        models[name] = model
    return model_scores

def deep_learning_hyper(model, model_grid, X_train, X_test, y_train, y_test):
    '''
    Fit and Score the deep learning models for hyperparameter tuning
    '''    
    random_search = RandomizedSearchCV(model,
                                    param_distributions=model_grid,
                                    n_iter=50,
                                    cv=5,
                                    random_state=42,
                                    refit=True,
                                    n_jobs=-1,
                                    verbose=True)
    random_search.fit(X_train, y_train)
    random_search_preds= random_search.predict(X_test)
    model_score = scoring(y_test, random_search_preds)
    model_best_param = random_search.best_params_
    return model_score, model_best_param, random_search

## Model Experimentation

In [34]:
tmp = preprocess(df_train.copy())

In [40]:
lab_en = LabelEncoder()
tmp.loc[:,'Material'] = lab_en.fit_transform(tmp.loc[:,'Material'])
tmp.loc[:,'International'] = lab_en.fit_transform(tmp.loc[:,'International'])
tmp.loc[:,'Express Shipment'] = lab_en.fit_transform(tmp.loc[:,'Express Shipment'])
tmp.loc[:,'Installation Included'] = lab_en.fit_transform(tmp.loc[:,'Installation Included'])
tmp.loc[:,'Transport'] = lab_en.fit_transform(tmp.loc[:,'Transport'])
tmp.loc[:,'Fragile'] = lab_en.fit_transform(tmp.loc[:,'Fragile'])
tmp.loc[:,'Customer Information'] = lab_en.fit_transform(tmp.loc[:,'Customer Information'])
tmp.loc[:,'Remote Location'] = lab_en.fit_transform(tmp.loc[:,'Remote Location'])
tmp.loc[:,'Province'] = lab_en.fit_transform(tmp.loc[:,'Province'])
tmp.loc[:,'Place'] = lab_en.fit_transform(tmp.loc[:,'Place'])

# test_data.loc[:,'Material'] = lab_en.fit_transform(test_data.loc[:,'Material'])
# test_data.loc[:,'International'] = lab_en.fit_transform(test_data.loc[:,'International'])
# test_data.loc[:,'Express Shipment'] = lab_en.fit_transform(test_data.loc[:,'Express Shipment'])
# test_data.loc[:,'Installation Included'] = lab_en.fit_transform(test_data.loc[:,'Installation Included'])
# test_data.loc[:,'Transport'] = lab_en.fit_transform(test_data.loc[:,'Transport'])
# test_data.loc[:,'Fragile'] = lab_en.fit_transform(test_data.loc[:,'Fragile'])
# test_data.loc[:,'Customer Information'] = lab_en.fit_transform(test_data.loc[:,'Customer Information'])
# test_data.loc[:,'Remote Location'] = lab_en.fit_transform(test_data.loc[:,'Remote Location'])

In [41]:
tmp

,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,...,Remote Location,scheduledYear,scheduledMonth,scheduledDay,deliveryYear,deliveryMonth,deliveryDay,Province,Pincode,Place
0,0.26,17.0,6.0,4128.0,1,13.91,16.27,1,1,0,...,0,2015.0,6.0,7.0,2015.0,6.0,3.0,38,50777.0,3345
1,0.28,3.0,3.0,61.0,1,6.83,15.00,0,0,0,...,0,2017.0,3.0,6.0,2017.0,3.0,5.0,53,12072.0,3341
2,0.07,8.0,5.0,237.0,3,4.96,21.18,0,0,0,...,1,2015.0,3.0,9.0,2015.0,3.0,8.0,50,19241.0,486
3,0.12,9.0,8.0,3102.0,0,5.81,16.31,0,0,0,...,1,2015.0,5.0,24.0,2015.0,5.0,20.0,20,86375.0,1423
4,0.15,17.0,6.0,324.0,0,3.18,11.94,1,1,1,...,0,2016.0,12.0,18.0,2016.0,12.0,14.0,41,52793.0,373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,0.37,37.0,10.0,16551.0,1,28.28,38.46,1,1,0,...,1,2018.0,3.0,28.0,2018.0,3.0,25.0,49,85335.0,3379
6496,0.67,15.0,8.0,18981.0,1,67.18,27.72,0,0,0,...,0,2015.0,8.0,29.0,2015.0,8.0,27.0,48,35766.0,3269
6497,0.68,19.0,8.0,587.0,3,6.92,10.38,1,0,0,...,0,2019.0,4.0,10.0,2019.0,4.0,9.0,22,80823.0,2554
6498,0.02,33.0,9.0,1269377.0,5,2929.13,69.76,0,1,0,...,1,2019.0,3.0,10.0,2019.0,3.0,12.0,47,35006.0,2010


In [50]:
model_tmp = CatBoostRegressor(use_best_model=True, eval_metric='RMSE')
model_tmp.fit(tmp, log1p(target))

CatBoostError: To employ param {'use_best_model': True} provide non-empty 'eval_set'.

In [27]:
df_train_copy = df_train.copy()
X = df_train_copy
y = log1p(target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = preprocess(X_train)
X_test = preprocess(X_test)

<ipython-input-26-1c166658cedd>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['scheduledYear'] = data['Scheduled Date'].dt.year
<ipython-input-26-1c166658cedd>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['scheduledMonth'] = data['Scheduled Date'].dt.month
<ipython-input-26-1c166658cedd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [43]:
test_data = preprocess(df_test.copy())
test_data.loc[:,'Material'] = lab_en.fit_transform(test_data.loc[:,'Material'])
test_data.loc[:,'International'] = lab_en.fit_transform(test_data.loc[:,'International'])
test_data.loc[:,'Express Shipment'] = lab_en.fit_transform(test_data.loc[:,'Express Shipment'])
test_data.loc[:,'Installation Included'] = lab_en.fit_transform(test_data.loc[:,'Installation Included'])
test_data.loc[:,'Transport'] = lab_en.fit_transform(test_data.loc[:,'Transport'])
test_data.loc[:,'Fragile'] = lab_en.fit_transform(test_data.loc[:,'Fragile'])
test_data.loc[:,'Customer Information'] = lab_en.fit_transform(test_data.loc[:,'Customer Information'])
test_data.loc[:,'Remote Location'] = lab_en.fit_transform(test_data.loc[:,'Remote Location'])
test_data.loc[:,'Province'] = lab_en.fit_transform(test_data.loc[:,'Province'])
test_data.loc[:,'Place'] = lab_en.fit_transform(test_data.loc[:,'Place'])

In [44]:
test_data

,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,...,Remote Location,scheduledYear,scheduledMonth,scheduledDay,deliveryYear,deliveryMonth,deliveryDay,Province,Pincode,Place
0,0.35,53.0,18.0,871.0,6,5.98,19.11,1,1,0,...,0,2017.0,7.0,3.0,2017.0,7.0,6.0,15,63481.0,2503
1,0.67,7.0,4.0,108.0,3,6.92,13.96,0,0,0,...,0,2016.0,5.0,2.0,2016.0,5.0,2.0,38,98253.0,854
2,0.61,6.0,5.0,97.0,0,4.23,13.62,1,0,0,...,0,2018.0,1.0,4.0,2018.0,1.0,6.0,4,83453.0,107
3,0.14,15.0,8.0,757.0,3,6.28,23.79,0,1,0,...,0,2017.0,9.0,14.0,2017.0,9.0,17.0,3,54778.0,195
4,0.63,10.0,4.0,1673.0,4,4.39,17.83,0,1,1,...,1,2017.0,12.0,3.0,2017.0,12.0,2.0,21,3040.0,1419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,0.36,13.0,6.0,23511.0,4,78.01,88.70,0,1,0,...,0,2016.0,2.0,9.0,2016.0,2.0,9.0,15,22615.0,2576
3496,0.72,29.0,14.0,2929.0,2,214.85,80.28,1,1,0,...,0,2018.0,11.0,26.0,2018.0,11.0,30.0,32,65012.0,1308
3497,0.59,12.0,6.0,2929.0,1,8.91,19.58,0,0,1,...,0,2017.0,4.0,1.0,2017.0,4.0,2.0,3,95571.0,1384
3498,0.45,17.0,9.0,565.0,0,6.75,10.89,0,1,1,...,0,2017.0,6.0,28.0,2017.0,6.0,28.0,47,49761.0,3117


In [45]:
submission(model_tmp, test_data, 'testing.csv')

,Customer Id,Cost
0,fffe3400310033003300,257.400818
1,fffe3600350035003400,273.255186
2,fffe3700360030003500,176.538517
3,fffe350038003600,202.852027
4,fffe3500390032003500,320.079869
...,...,...
3495,fffe3200300037003100,1219.230370
3496,fffe390031003100,3114.003588
3497,fffe3200350031003700,661.137439
3498,fffe3100380031003000,268.579273


In [28]:
X_test

,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,...,Remote Location,scheduledYear,scheduledMonth,scheduledDay,deliveryYear,deliveryMonth,deliveryDay,Province,Pincode,Place
3106,0.70,21.0,13.0,2453.0,4,11.52,17.84,1,1,2,...,1,2016.0,7.0,19.0,2016.0,7.0,21.0,27,11042.0,1053
6161,0.14,15.0,8.0,326.0,1,6.54,22.83,1,1,2,...,1,2018.0,4.0,24.0,2018.0,4.0,20.0,38,41609.0,710
1867,0.69,19.0,7.0,230.0,7,6.83,23.71,1,1,1,...,1,2018.0,4.0,20.0,2018.0,4.0,24.0,7,27946.0,557
3238,0.52,11.0,6.0,18765.0,2,26.38,63.68,2,1,1,...,1,2017.0,1.0,21.0,2017.0,1.0,21.0,14,21999.0,514
5509,0.46,11.0,8.0,421.0,4,3.26,16.42,1,1,2,...,1,2015.0,7.0,6.0,2015.0,7.0,4.0,35,16111.0,282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,0.25,12.0,8.0,414.0,1,4.04,18.80,1,1,2,...,1,2018.0,7.0,1.0,2018.0,7.0,5.0,4,34600.0,760
1608,0.67,17.0,9.0,8069.0,2,38.13,97.67,1,2,2,...,1,2015.0,11.0,29.0,2015.0,11.0,29.0,28,99743.0,919
2191,0.90,13.0,7.0,867.0,4,5.60,22.93,2,1,2,...,1,2016.0,1.0,18.0,2016.0,1.0,17.0,7,43000.0,894
624,0.46,25.0,7.0,11914.0,2,7.64,69.06,1,1,1,...,1,2015.0,1.0,12.0,2015.0,1.0,9.0,34,9519.0,158


In [29]:
# import lightgbm as lgb
# param = {'objective': 'regression',
#          'boosting': 'gbdt',  
#          'metric': 'l2_root',
#          'learning_rate': 0.01, 
#          'num_iterations': 10000,
#          'min_data_in_leaf': 15,
#          'bagging_fraction': 0.90,
#          'bagging_freq': 2,
#          'bagging_seed': 3,
#          'feature_fraction': 0.99,
#          'feature_fraction_seed': 2,
#          'early_stopping_round': 2000,
#          'max_bin': 500
#          }
# gbm = lgb.LGBMRegressor(**param)
# gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)],
#         eval_metric='rmse',
#         early_stopping_rounds = 100)

In [30]:
# scoring(y_test, gbm.predict(X_test))

In [31]:
deep_models = {'XGB': xgb.XGBRegressor(),
               'GBR': GradientBoostingRegressor(),
               'ADA': AdaBoostRegressor(),
               'CAT': CatBoostRegressor(),
               'XGBRF': xgb.XGBRFRegressor()}

deep_model_scores = deep_learning(deep_models, X_train, X_test, y_train, y_test)

Learning rate set to 0.052342
0:	learn: 1.5808147	total: 2.75ms	remaining: 2.75s
1:	learn: 1.5149876	total: 4.66ms	remaining: 2.33s
2:	learn: 1.4565029	total: 6.51ms	remaining: 2.16s
3:	learn: 1.3983995	total: 8.3ms	remaining: 2.07s
4:	learn: 1.3434940	total: 10.1ms	remaining: 2s
5:	learn: 1.2897454	total: 11.9ms	remaining: 1.98s
6:	learn: 1.2406718	total: 13.8ms	remaining: 1.96s
7:	learn: 1.1934613	total: 15.9ms	remaining: 1.97s
8:	learn: 1.1485243	total: 18ms	remaining: 1.98s
9:	learn: 1.1054225	total: 19.7ms	remaining: 1.95s
10:	learn: 1.0638534	total: 21.6ms	remaining: 1.94s
11:	learn: 1.0249526	total: 23.5ms	remaining: 1.93s
12:	learn: 0.9891051	total: 25.3ms	remaining: 1.92s
13:	learn: 0.9547650	total: 27.1ms	remaining: 1.91s
14:	learn: 0.9234792	total: 29ms	remaining: 1.9s
15:	learn: 0.8929270	total: 31ms	remaining: 1.9s
16:	learn: 0.8654199	total: 32.9ms	remaining: 1.9s
17:	learn: 0.8396707	total: 34.8ms	remaining: 1.9s
18:	learn: 0.8136949	total: 36.6ms	remaining: 1.89s
19:	le

177:	learn: 0.3054057	total: 334ms	remaining: 1.54s
178:	learn: 0.3048623	total: 336ms	remaining: 1.54s
179:	learn: 0.3045367	total: 338ms	remaining: 1.54s
180:	learn: 0.3038488	total: 340ms	remaining: 1.54s
181:	learn: 0.3033731	total: 342ms	remaining: 1.54s
182:	learn: 0.3028263	total: 344ms	remaining: 1.54s
183:	learn: 0.3024732	total: 346ms	remaining: 1.53s
184:	learn: 0.3018272	total: 348ms	remaining: 1.53s
185:	learn: 0.3016203	total: 350ms	remaining: 1.53s
186:	learn: 0.3011321	total: 352ms	remaining: 1.53s
187:	learn: 0.3008581	total: 355ms	remaining: 1.53s
188:	learn: 0.3002673	total: 358ms	remaining: 1.53s
189:	learn: 0.2989133	total: 360ms	remaining: 1.53s
190:	learn: 0.2978342	total: 361ms	remaining: 1.53s
191:	learn: 0.2970641	total: 364ms	remaining: 1.53s
192:	learn: 0.2965524	total: 366ms	remaining: 1.53s
193:	learn: 0.2960621	total: 368ms	remaining: 1.53s
194:	learn: 0.2949250	total: 370ms	remaining: 1.53s
195:	learn: 0.2944445	total: 372ms	remaining: 1.52s
196:	learn: 

356:	learn: 0.2317277	total: 674ms	remaining: 1.21s
357:	learn: 0.2315629	total: 676ms	remaining: 1.21s
358:	learn: 0.2312051	total: 678ms	remaining: 1.21s
359:	learn: 0.2308830	total: 680ms	remaining: 1.21s
360:	learn: 0.2306718	total: 682ms	remaining: 1.21s
361:	learn: 0.2302837	total: 684ms	remaining: 1.21s
362:	learn: 0.2299365	total: 686ms	remaining: 1.2s
363:	learn: 0.2297830	total: 688ms	remaining: 1.2s
364:	learn: 0.2296613	total: 690ms	remaining: 1.2s
365:	learn: 0.2294108	total: 692ms	remaining: 1.2s
366:	learn: 0.2291744	total: 694ms	remaining: 1.2s
367:	learn: 0.2288201	total: 695ms	remaining: 1.19s
368:	learn: 0.2286516	total: 697ms	remaining: 1.19s
369:	learn: 0.2284334	total: 699ms	remaining: 1.19s
370:	learn: 0.2282775	total: 701ms	remaining: 1.19s
371:	learn: 0.2279062	total: 703ms	remaining: 1.19s
372:	learn: 0.2277274	total: 704ms	remaining: 1.18s
373:	learn: 0.2274486	total: 706ms	remaining: 1.18s
374:	learn: 0.2272600	total: 708ms	remaining: 1.18s
375:	learn: 0.227

532:	learn: 0.2009772	total: 1.01s	remaining: 889ms
533:	learn: 0.2008672	total: 1.02s	remaining: 887ms
534:	learn: 0.2007323	total: 1.02s	remaining: 885ms
535:	learn: 0.2005567	total: 1.02s	remaining: 884ms
536:	learn: 0.2004108	total: 1.02s	remaining: 882ms
537:	learn: 0.2003131	total: 1.02s	remaining: 880ms
538:	learn: 0.2001245	total: 1.03s	remaining: 878ms
539:	learn: 0.2000067	total: 1.03s	remaining: 876ms
540:	learn: 0.1998732	total: 1.03s	remaining: 875ms
541:	learn: 0.1997270	total: 1.03s	remaining: 873ms
542:	learn: 0.1995640	total: 1.03s	remaining: 872ms
543:	learn: 0.1993479	total: 1.04s	remaining: 870ms
544:	learn: 0.1992453	total: 1.04s	remaining: 869ms
545:	learn: 0.1991556	total: 1.04s	remaining: 867ms
546:	learn: 0.1990729	total: 1.04s	remaining: 866ms
547:	learn: 0.1988784	total: 1.05s	remaining: 864ms
548:	learn: 0.1987603	total: 1.05s	remaining: 862ms
549:	learn: 0.1986174	total: 1.05s	remaining: 861ms
550:	learn: 0.1985303	total: 1.05s	remaining: 859ms
551:	learn: 

697:	learn: 0.1797863	total: 1.35s	remaining: 583ms
698:	learn: 0.1797077	total: 1.35s	remaining: 581ms
699:	learn: 0.1795859	total: 1.35s	remaining: 579ms
700:	learn: 0.1794845	total: 1.35s	remaining: 577ms
701:	learn: 0.1793933	total: 1.35s	remaining: 576ms
702:	learn: 0.1792852	total: 1.36s	remaining: 574ms
703:	learn: 0.1792060	total: 1.36s	remaining: 572ms
704:	learn: 0.1790959	total: 1.36s	remaining: 571ms
705:	learn: 0.1789066	total: 1.36s	remaining: 569ms
706:	learn: 0.1788386	total: 1.37s	remaining: 567ms
707:	learn: 0.1787824	total: 1.37s	remaining: 565ms
708:	learn: 0.1786303	total: 1.37s	remaining: 563ms
709:	learn: 0.1783603	total: 1.37s	remaining: 561ms
710:	learn: 0.1782498	total: 1.38s	remaining: 560ms
711:	learn: 0.1781309	total: 1.38s	remaining: 558ms
712:	learn: 0.1779565	total: 1.38s	remaining: 556ms
713:	learn: 0.1778986	total: 1.38s	remaining: 554ms
714:	learn: 0.1777043	total: 1.39s	remaining: 552ms
715:	learn: 0.1776298	total: 1.39s	remaining: 550ms
716:	learn: 

908:	learn: 0.1590595	total: 1.85s	remaining: 185ms
909:	learn: 0.1589596	total: 1.85s	remaining: 183ms
910:	learn: 0.1588334	total: 1.85s	remaining: 181ms
911:	learn: 0.1588104	total: 1.86s	remaining: 179ms
912:	learn: 0.1587658	total: 1.86s	remaining: 177ms
913:	learn: 0.1586967	total: 1.86s	remaining: 175ms
914:	learn: 0.1586214	total: 1.86s	remaining: 173ms
915:	learn: 0.1585681	total: 1.87s	remaining: 171ms
916:	learn: 0.1584867	total: 1.87s	remaining: 169ms
917:	learn: 0.1583738	total: 1.87s	remaining: 167ms
918:	learn: 0.1583169	total: 1.88s	remaining: 165ms
919:	learn: 0.1581746	total: 1.88s	remaining: 163ms
920:	learn: 0.1580275	total: 1.88s	remaining: 161ms
921:	learn: 0.1579397	total: 1.88s	remaining: 159ms
922:	learn: 0.1578470	total: 1.88s	remaining: 157ms
923:	learn: 0.1577214	total: 1.89s	remaining: 155ms
924:	learn: 0.1576494	total: 1.89s	remaining: 153ms
925:	learn: 0.1575535	total: 1.89s	remaining: 151ms
926:	learn: 0.1574480	total: 1.89s	remaining: 149ms
927:	learn: 

In [32]:
deep_model_scores

{'XGB': 99.85638465618919,
 'GBR': 99.82708400388037,
 'ADA': 99.35969487060522,
 'CAT': 99.89750104328122,
 'XGBRF': 99.75632966380104}

In [19]:
#model_grid = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
              #'n_estimators':[100,250,500,750,1000,1250,1500,1750]}
model = CatBoostRegressor()
#model_score, model_best_param, random_search = deep_learning_hyper(model, model_grid, X_train, X_test, y_train, y_test)

In [20]:
#X = preprocess(X)
model.fit(X, y)

Learning rate set to 0.054463
0:	learn: 1.5775172	total: 5.02ms	remaining: 5.01s
1:	learn: 1.5088861	total: 7.58ms	remaining: 3.78s
2:	learn: 1.4479873	total: 9.83ms	remaining: 3.27s
3:	learn: 1.3856388	total: 11.8ms	remaining: 2.94s
4:	learn: 1.3255207	total: 13.8ms	remaining: 2.75s
5:	learn: 1.2701237	total: 15.8ms	remaining: 2.62s
6:	learn: 1.2178510	total: 17.9ms	remaining: 2.54s
7:	learn: 1.1694541	total: 20.1ms	remaining: 2.49s
8:	learn: 1.1225888	total: 22.1ms	remaining: 2.44s
9:	learn: 1.0782891	total: 24.2ms	remaining: 2.4s
10:	learn: 1.0364480	total: 26.2ms	remaining: 2.35s
11:	learn: 0.9988463	total: 28.1ms	remaining: 2.31s
12:	learn: 0.9627557	total: 30.1ms	remaining: 2.29s
13:	learn: 0.9293330	total: 32.1ms	remaining: 2.26s
14:	learn: 0.8978509	total: 34.6ms	remaining: 2.27s
15:	learn: 0.8670983	total: 37.1ms	remaining: 2.28s
16:	learn: 0.8378712	total: 39.1ms	remaining: 2.26s
17:	learn: 0.8117718	total: 41.1ms	remaining: 2.24s
18:	learn: 0.7874705	total: 43.1ms	remaining:

169:	learn: 0.3077349	total: 339ms	remaining: 1.66s
170:	learn: 0.3071523	total: 341ms	remaining: 1.65s
171:	learn: 0.3062812	total: 343ms	remaining: 1.65s
172:	learn: 0.3052824	total: 345ms	remaining: 1.65s
173:	learn: 0.3046008	total: 347ms	remaining: 1.65s
174:	learn: 0.3037921	total: 349ms	remaining: 1.65s
175:	learn: 0.3029597	total: 351ms	remaining: 1.65s
176:	learn: 0.3023713	total: 354ms	remaining: 1.64s
177:	learn: 0.3018805	total: 356ms	remaining: 1.64s
178:	learn: 0.3011523	total: 358ms	remaining: 1.64s
179:	learn: 0.3005724	total: 361ms	remaining: 1.64s
180:	learn: 0.2997394	total: 363ms	remaining: 1.64s
181:	learn: 0.2990391	total: 365ms	remaining: 1.64s
182:	learn: 0.2984749	total: 367ms	remaining: 1.64s
183:	learn: 0.2977208	total: 369ms	remaining: 1.64s
184:	learn: 0.2972613	total: 371ms	remaining: 1.64s
185:	learn: 0.2966792	total: 374ms	remaining: 1.63s
186:	learn: 0.2961388	total: 376ms	remaining: 1.63s
187:	learn: 0.2958395	total: 378ms	remaining: 1.63s
188:	learn: 

345:	learn: 0.2380368	total: 680ms	remaining: 1.28s
346:	learn: 0.2377521	total: 682ms	remaining: 1.28s
347:	learn: 0.2375706	total: 684ms	remaining: 1.28s
348:	learn: 0.2373690	total: 686ms	remaining: 1.28s
349:	learn: 0.2370380	total: 688ms	remaining: 1.28s
350:	learn: 0.2367971	total: 691ms	remaining: 1.28s
351:	learn: 0.2364609	total: 693ms	remaining: 1.27s
352:	learn: 0.2361854	total: 695ms	remaining: 1.27s
353:	learn: 0.2358239	total: 696ms	remaining: 1.27s
354:	learn: 0.2356715	total: 699ms	remaining: 1.27s
355:	learn: 0.2354503	total: 700ms	remaining: 1.27s
356:	learn: 0.2352205	total: 702ms	remaining: 1.26s
357:	learn: 0.2351167	total: 704ms	remaining: 1.26s
358:	learn: 0.2349400	total: 706ms	remaining: 1.26s
359:	learn: 0.2347702	total: 708ms	remaining: 1.26s
360:	learn: 0.2346549	total: 710ms	remaining: 1.26s
361:	learn: 0.2344946	total: 712ms	remaining: 1.25s
362:	learn: 0.2343171	total: 714ms	remaining: 1.25s
363:	learn: 0.2340469	total: 716ms	remaining: 1.25s
364:	learn: 

524:	learn: 0.2064767	total: 1.02s	remaining: 925ms
525:	learn: 0.2063525	total: 1.02s	remaining: 923ms
526:	learn: 0.2062457	total: 1.03s	remaining: 921ms
527:	learn: 0.2061131	total: 1.03s	remaining: 919ms
528:	learn: 0.2059798	total: 1.03s	remaining: 917ms
529:	learn: 0.2058570	total: 1.03s	remaining: 915ms
530:	learn: 0.2057685	total: 1.03s	remaining: 913ms
531:	learn: 0.2056596	total: 1.03s	remaining: 911ms
532:	learn: 0.2055297	total: 1.04s	remaining: 910ms
533:	learn: 0.2053946	total: 1.04s	remaining: 908ms
534:	learn: 0.2052576	total: 1.04s	remaining: 906ms
535:	learn: 0.2050953	total: 1.04s	remaining: 904ms
536:	learn: 0.2049735	total: 1.04s	remaining: 902ms
537:	learn: 0.2048111	total: 1.05s	remaining: 900ms
538:	learn: 0.2046889	total: 1.05s	remaining: 898ms
539:	learn: 0.2045416	total: 1.05s	remaining: 896ms
540:	learn: 0.2044389	total: 1.05s	remaining: 894ms
541:	learn: 0.2043136	total: 1.06s	remaining: 892ms
542:	learn: 0.2041846	total: 1.06s	remaining: 891ms
543:	learn: 

742:	learn: 0.1815922	total: 1.52s	remaining: 526ms
743:	learn: 0.1814670	total: 1.52s	remaining: 524ms
744:	learn: 0.1813756	total: 1.53s	remaining: 523ms
745:	learn: 0.1812985	total: 1.53s	remaining: 521ms
746:	learn: 0.1811814	total: 1.53s	remaining: 519ms
747:	learn: 0.1810423	total: 1.54s	remaining: 521ms
748:	learn: 0.1809776	total: 1.55s	remaining: 519ms
749:	learn: 0.1808838	total: 1.55s	remaining: 517ms
750:	learn: 0.1808097	total: 1.55s	remaining: 514ms
751:	learn: 0.1807451	total: 1.55s	remaining: 512ms
752:	learn: 0.1806630	total: 1.56s	remaining: 510ms
753:	learn: 0.1805717	total: 1.56s	remaining: 508ms
754:	learn: 0.1804868	total: 1.56s	remaining: 506ms
755:	learn: 0.1804014	total: 1.56s	remaining: 504ms
756:	learn: 0.1803316	total: 1.56s	remaining: 502ms
757:	learn: 0.1802666	total: 1.57s	remaining: 500ms
758:	learn: 0.1801547	total: 1.57s	remaining: 498ms
759:	learn: 0.1800727	total: 1.57s	remaining: 496ms
760:	learn: 0.1799878	total: 1.57s	remaining: 494ms
761:	learn: 

985:	learn: 0.1595256	total: 2.02s	remaining: 28.7ms
986:	learn: 0.1594361	total: 2.03s	remaining: 26.7ms
987:	learn: 0.1593450	total: 2.03s	remaining: 24.6ms
988:	learn: 0.1592612	total: 2.03s	remaining: 22.6ms
989:	learn: 0.1591971	total: 2.03s	remaining: 20.5ms
990:	learn: 0.1591318	total: 2.03s	remaining: 18.5ms
991:	learn: 0.1590112	total: 2.04s	remaining: 16.4ms
992:	learn: 0.1589303	total: 2.04s	remaining: 14.4ms
993:	learn: 0.1588546	total: 2.04s	remaining: 12.3ms
994:	learn: 0.1587596	total: 2.04s	remaining: 10.3ms
995:	learn: 0.1586466	total: 2.04s	remaining: 8.21ms
996:	learn: 0.1585918	total: 2.05s	remaining: 6.16ms
997:	learn: 0.1584969	total: 2.05s	remaining: 4.11ms
998:	learn: 0.1583839	total: 2.05s	remaining: 2.05ms
999:	learn: 0.1582906	total: 2.05s	remaining: 0us


In [21]:
submission_df = submission(model, df_test, 'CAT_1.csv')

In [22]:
submission_df

,Customer Id,Cost
0,fffe3400310033003300,257.400818
1,fffe3600350035003400,273.255186
2,fffe3700360030003500,176.538517
3,fffe350038003600,202.852027
4,fffe3500390032003500,320.079869
...,...,...
3495,fffe3200300037003100,1219.230370
3496,fffe390031003100,3114.003588
3497,fffe3200350031003700,661.137439
3498,fffe3100380031003000,268.579273


In [64]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
estimators = [
        ('RFC' ,RandomForestRegressor()),
        ('LR',  LinearRegression()),
        ('DTC', DecisionTreeRegressor()),
        ('SVC', AdaBoostRegressor()),
        ('CAT', xgb.XGBRFRegressor()),
        ('GBR', GradientBoostingRegressor())
        
]

clf = StackingRegressor(
    estimators=estimators, 
    final_estimator= CatBoostRegressor())

clf.fit(X_train, y_train)

Learning rate set to 0.052342
0:	learn: 1.5750707	total: 2.35ms	remaining: 2.35s
1:	learn: 1.5030492	total: 4.22ms	remaining: 2.11s
2:	learn: 1.4360635	total: 5.97ms	remaining: 1.99s
3:	learn: 1.3703969	total: 7.73ms	remaining: 1.93s
4:	learn: 1.3082630	total: 9.5ms	remaining: 1.89s
5:	learn: 1.2507385	total: 11.3ms	remaining: 1.86s
6:	learn: 1.1961423	total: 13ms	remaining: 1.85s
7:	learn: 1.1444137	total: 14.9ms	remaining: 1.84s
8:	learn: 1.0960354	total: 16.7ms	remaining: 1.84s
9:	learn: 1.0493864	total: 18.5ms	remaining: 1.83s
10:	learn: 1.0052030	total: 20.2ms	remaining: 1.81s
11:	learn: 0.9629691	total: 22ms	remaining: 1.81s
12:	learn: 0.9228927	total: 23.8ms	remaining: 1.8s
13:	learn: 0.8854343	total: 25.8ms	remaining: 1.82s
14:	learn: 0.8504100	total: 28ms	remaining: 1.84s
15:	learn: 0.8165858	total: 29.9ms	remaining: 1.84s
16:	learn: 0.7853249	total: 32.2ms	remaining: 1.86s
17:	learn: 0.7554251	total: 34.1ms	remaining: 1.86s
18:	learn: 0.7275428	total: 35.8ms	remaining: 1.85s


192:	learn: 0.3096242	total: 327ms	remaining: 1.37s
193:	learn: 0.3094520	total: 329ms	remaining: 1.37s
194:	learn: 0.3092922	total: 331ms	remaining: 1.36s
195:	learn: 0.3090825	total: 332ms	remaining: 1.36s
196:	learn: 0.3088357	total: 334ms	remaining: 1.36s
197:	learn: 0.3087555	total: 336ms	remaining: 1.36s
198:	learn: 0.3086327	total: 337ms	remaining: 1.36s
199:	learn: 0.3084005	total: 339ms	remaining: 1.35s
200:	learn: 0.3082440	total: 341ms	remaining: 1.35s
201:	learn: 0.3080473	total: 342ms	remaining: 1.35s
202:	learn: 0.3077380	total: 344ms	remaining: 1.35s
203:	learn: 0.3074101	total: 346ms	remaining: 1.35s
204:	learn: 0.3070939	total: 347ms	remaining: 1.35s
205:	learn: 0.3068670	total: 349ms	remaining: 1.35s
206:	learn: 0.3066054	total: 351ms	remaining: 1.34s
207:	learn: 0.3064986	total: 353ms	remaining: 1.34s
208:	learn: 0.3063203	total: 355ms	remaining: 1.34s
209:	learn: 0.3061259	total: 357ms	remaining: 1.34s
210:	learn: 0.3058076	total: 358ms	remaining: 1.34s
211:	learn: 

400:	learn: 0.2733289	total: 666ms	remaining: 994ms
401:	learn: 0.2731538	total: 667ms	remaining: 993ms
402:	learn: 0.2730421	total: 669ms	remaining: 991ms
403:	learn: 0.2729134	total: 671ms	remaining: 990ms
404:	learn: 0.2726311	total: 673ms	remaining: 988ms
405:	learn: 0.2725480	total: 674ms	remaining: 986ms
406:	learn: 0.2723722	total: 676ms	remaining: 985ms
407:	learn: 0.2722894	total: 677ms	remaining: 983ms
408:	learn: 0.2720851	total: 679ms	remaining: 982ms
409:	learn: 0.2718595	total: 681ms	remaining: 980ms
410:	learn: 0.2716662	total: 683ms	remaining: 978ms
411:	learn: 0.2715173	total: 684ms	remaining: 977ms
412:	learn: 0.2714357	total: 686ms	remaining: 975ms
413:	learn: 0.2713080	total: 688ms	remaining: 974ms
414:	learn: 0.2711393	total: 690ms	remaining: 972ms
415:	learn: 0.2710016	total: 691ms	remaining: 970ms
416:	learn: 0.2707975	total: 693ms	remaining: 969ms
417:	learn: 0.2706384	total: 695ms	remaining: 967ms
418:	learn: 0.2705583	total: 696ms	remaining: 965ms
419:	learn: 

601:	learn: 0.2503450	total: 1s	remaining: 663ms
602:	learn: 0.2502050	total: 1s	remaining: 661ms
603:	learn: 0.2501387	total: 1.01s	remaining: 660ms
604:	learn: 0.2499959	total: 1.01s	remaining: 658ms
605:	learn: 0.2499728	total: 1.01s	remaining: 656ms
606:	learn: 0.2498965	total: 1.01s	remaining: 655ms
607:	learn: 0.2498201	total: 1.01s	remaining: 653ms
608:	learn: 0.2496898	total: 1.01s	remaining: 652ms
609:	learn: 0.2495654	total: 1.02s	remaining: 650ms
610:	learn: 0.2493860	total: 1.02s	remaining: 649ms
611:	learn: 0.2492806	total: 1.02s	remaining: 647ms
612:	learn: 0.2492361	total: 1.02s	remaining: 646ms
613:	learn: 0.2491289	total: 1.02s	remaining: 644ms
614:	learn: 0.2489933	total: 1.03s	remaining: 643ms
615:	learn: 0.2489058	total: 1.03s	remaining: 641ms
616:	learn: 0.2488434	total: 1.03s	remaining: 639ms
617:	learn: 0.2487014	total: 1.03s	remaining: 638ms
618:	learn: 0.2485873	total: 1.03s	remaining: 636ms
619:	learn: 0.2484635	total: 1.03s	remaining: 634ms
620:	learn: 0.2482

791:	learn: 0.2340773	total: 1.34s	remaining: 351ms
792:	learn: 0.2340008	total: 1.34s	remaining: 349ms
793:	learn: 0.2339429	total: 1.34s	remaining: 348ms
794:	learn: 0.2338515	total: 1.34s	remaining: 346ms
795:	learn: 0.2337834	total: 1.34s	remaining: 344ms
796:	learn: 0.2337111	total: 1.34s	remaining: 343ms
797:	learn: 0.2336547	total: 1.35s	remaining: 341ms
798:	learn: 0.2336380	total: 1.35s	remaining: 339ms
799:	learn: 0.2335702	total: 1.35s	remaining: 338ms
800:	learn: 0.2334710	total: 1.35s	remaining: 336ms
801:	learn: 0.2333626	total: 1.35s	remaining: 334ms
802:	learn: 0.2333078	total: 1.35s	remaining: 333ms
803:	learn: 0.2332653	total: 1.36s	remaining: 331ms
804:	learn: 0.2331886	total: 1.36s	remaining: 329ms
805:	learn: 0.2331294	total: 1.36s	remaining: 328ms
806:	learn: 0.2330810	total: 1.36s	remaining: 326ms
807:	learn: 0.2329928	total: 1.36s	remaining: 324ms
808:	learn: 0.2329289	total: 1.36s	remaining: 322ms
809:	learn: 0.2328650	total: 1.37s	remaining: 321ms
810:	learn: 

990:	learn: 0.2207379	total: 1.67s	remaining: 15.2ms
991:	learn: 0.2207201	total: 1.68s	remaining: 13.5ms
992:	learn: 0.2206964	total: 1.68s	remaining: 11.8ms
993:	learn: 0.2206878	total: 1.68s	remaining: 10.1ms
994:	learn: 0.2206046	total: 1.68s	remaining: 8.45ms
995:	learn: 0.2205482	total: 1.68s	remaining: 6.76ms
996:	learn: 0.2204861	total: 1.69s	remaining: 5.07ms
997:	learn: 0.2204071	total: 1.69s	remaining: 3.38ms
998:	learn: 0.2203646	total: 1.69s	remaining: 1.69ms
999:	learn: 0.2203203	total: 1.69s	remaining: 0us


StackingRegressor(estimators=[('RFC', RandomForestRegressor()),
                              ('LR', LinearRegression()),
                              ('DTC', DecisionTreeRegressor()),
                              ('SVC', AdaBoostRegressor()),
                              ('CAT',
                               XGBRFRegressor(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bytree=None, gamma=None,
                                              gpu_id=None,
                                              importance_type='gain',
                                              interaction_constraints=None,
                                              max_delta_step=None,
                                              max_depth=N...
                                              min_child_weight=None,
                                              missing=nan,
                           

In [65]:
scoring(y_test, clf.predict(X_test))

99.85532113175722

In [63]:
clf.fit(X, y)
submission(clf, df_test, 'multi.csv')

Learning rate set to 0.054463
0:	learn: 1.5775172	total: 3.57ms	remaining: 3.56s
1:	learn: 1.5088861	total: 5.71ms	remaining: 2.85s
2:	learn: 1.4479873	total: 7.66ms	remaining: 2.54s
3:	learn: 1.3856388	total: 9.68ms	remaining: 2.41s
4:	learn: 1.3255207	total: 11.7ms	remaining: 2.32s
5:	learn: 1.2701237	total: 13.6ms	remaining: 2.26s
6:	learn: 1.2178510	total: 15.6ms	remaining: 2.21s
7:	learn: 1.1694541	total: 17.7ms	remaining: 2.19s
8:	learn: 1.1225888	total: 19.7ms	remaining: 2.17s
9:	learn: 1.0782891	total: 21.7ms	remaining: 2.15s
10:	learn: 1.0364480	total: 23.6ms	remaining: 2.13s
11:	learn: 0.9988463	total: 25.6ms	remaining: 2.1s
12:	learn: 0.9627557	total: 27.6ms	remaining: 2.1s
13:	learn: 0.9293330	total: 29.6ms	remaining: 2.08s
14:	learn: 0.8978509	total: 31.8ms	remaining: 2.09s
15:	learn: 0.8670983	total: 34ms	remaining: 2.09s
16:	learn: 0.8378712	total: 36.5ms	remaining: 2.11s
17:	learn: 0.8117718	total: 38.5ms	remaining: 2.1s
18:	learn: 0.7874705	total: 40.6ms	remaining: 2.1

170:	learn: 0.3071523	total: 336ms	remaining: 1.63s
171:	learn: 0.3062812	total: 338ms	remaining: 1.63s
172:	learn: 0.3052824	total: 340ms	remaining: 1.63s
173:	learn: 0.3046008	total: 342ms	remaining: 1.63s
174:	learn: 0.3037921	total: 344ms	remaining: 1.62s
175:	learn: 0.3029597	total: 346ms	remaining: 1.62s
176:	learn: 0.3023713	total: 348ms	remaining: 1.62s
177:	learn: 0.3018805	total: 351ms	remaining: 1.62s
178:	learn: 0.3011523	total: 353ms	remaining: 1.62s
179:	learn: 0.3005724	total: 355ms	remaining: 1.62s
180:	learn: 0.2997394	total: 357ms	remaining: 1.62s
181:	learn: 0.2990391	total: 359ms	remaining: 1.61s
182:	learn: 0.2984749	total: 361ms	remaining: 1.61s
183:	learn: 0.2977208	total: 364ms	remaining: 1.61s
184:	learn: 0.2972613	total: 366ms	remaining: 1.61s
185:	learn: 0.2966792	total: 368ms	remaining: 1.61s
186:	learn: 0.2961388	total: 370ms	remaining: 1.61s
187:	learn: 0.2958395	total: 372ms	remaining: 1.61s
188:	learn: 0.2953050	total: 374ms	remaining: 1.61s
189:	learn: 

346:	learn: 0.2377521	total: 676ms	remaining: 1.27s
347:	learn: 0.2375706	total: 678ms	remaining: 1.27s
348:	learn: 0.2373690	total: 680ms	remaining: 1.27s
349:	learn: 0.2370380	total: 682ms	remaining: 1.26s
350:	learn: 0.2367971	total: 684ms	remaining: 1.26s
351:	learn: 0.2364609	total: 685ms	remaining: 1.26s
352:	learn: 0.2361854	total: 688ms	remaining: 1.26s
353:	learn: 0.2358239	total: 690ms	remaining: 1.26s
354:	learn: 0.2356715	total: 692ms	remaining: 1.26s
355:	learn: 0.2354503	total: 694ms	remaining: 1.25s
356:	learn: 0.2352205	total: 696ms	remaining: 1.25s
357:	learn: 0.2351167	total: 697ms	remaining: 1.25s
358:	learn: 0.2349400	total: 699ms	remaining: 1.25s
359:	learn: 0.2347702	total: 701ms	remaining: 1.25s
360:	learn: 0.2346549	total: 703ms	remaining: 1.24s
361:	learn: 0.2344946	total: 705ms	remaining: 1.24s
362:	learn: 0.2343171	total: 707ms	remaining: 1.24s
363:	learn: 0.2340469	total: 709ms	remaining: 1.24s
364:	learn: 0.2338266	total: 710ms	remaining: 1.24s
365:	learn: 

528:	learn: 0.2059798	total: 1.02s	remaining: 906ms
529:	learn: 0.2058570	total: 1.02s	remaining: 905ms
530:	learn: 0.2057685	total: 1.02s	remaining: 903ms
531:	learn: 0.2056596	total: 1.02s	remaining: 901ms
532:	learn: 0.2055297	total: 1.02s	remaining: 899ms
533:	learn: 0.2053946	total: 1.03s	remaining: 897ms
534:	learn: 0.2052576	total: 1.04s	remaining: 902ms
535:	learn: 0.2050953	total: 1.04s	remaining: 900ms
536:	learn: 0.2049735	total: 1.04s	remaining: 898ms
537:	learn: 0.2048111	total: 1.04s	remaining: 896ms
538:	learn: 0.2046889	total: 1.04s	remaining: 894ms
539:	learn: 0.2045416	total: 1.05s	remaining: 892ms
540:	learn: 0.2044389	total: 1.05s	remaining: 890ms
541:	learn: 0.2043136	total: 1.05s	remaining: 888ms
542:	learn: 0.2041846	total: 1.05s	remaining: 886ms
543:	learn: 0.2040640	total: 1.05s	remaining: 884ms
544:	learn: 0.2039381	total: 1.06s	remaining: 882ms
545:	learn: 0.2038016	total: 1.06s	remaining: 880ms
546:	learn: 0.2036074	total: 1.06s	remaining: 878ms
547:	learn: 

701:	learn: 0.1855506	total: 1.36s	remaining: 577ms
702:	learn: 0.1854786	total: 1.36s	remaining: 575ms
703:	learn: 0.1853762	total: 1.36s	remaining: 573ms
704:	learn: 0.1852804	total: 1.36s	remaining: 571ms
705:	learn: 0.1851370	total: 1.37s	remaining: 569ms
706:	learn: 0.1850619	total: 1.37s	remaining: 567ms
707:	learn: 0.1849684	total: 1.37s	remaining: 566ms
708:	learn: 0.1848598	total: 1.37s	remaining: 564ms
709:	learn: 0.1847462	total: 1.38s	remaining: 562ms
710:	learn: 0.1846194	total: 1.38s	remaining: 560ms
711:	learn: 0.1845362	total: 1.38s	remaining: 558ms
712:	learn: 0.1844064	total: 1.38s	remaining: 556ms
713:	learn: 0.1843290	total: 1.38s	remaining: 554ms
714:	learn: 0.1842539	total: 1.38s	remaining: 552ms
715:	learn: 0.1840596	total: 1.39s	remaining: 550ms
716:	learn: 0.1839337	total: 1.39s	remaining: 548ms
717:	learn: 0.1838540	total: 1.39s	remaining: 546ms
718:	learn: 0.1837868	total: 1.39s	remaining: 544ms
719:	learn: 0.1837072	total: 1.39s	remaining: 542ms
720:	learn: 

...

455:	learn: 0.2105468	total: 917ms	remaining: 1.09s
456:	learn: 0.2103046	total: 919ms	remaining: 1.09s
457:	learn: 0.2100526	total: 921ms	remaining: 1.09s
458:	learn: 0.2098829	total: 923ms	remaining: 1.09s
459:	learn: 0.2097095	total: 924ms	remaining: 1.08s
460:	learn: 0.2094387	total: 926ms	remaining: 1.08s
461:	learn: 0.2092949	total: 928ms	remaining: 1.08s
462:	learn: 0.2090967	total: 930ms	remaining: 1.08s
463:	learn: 0.2089840	total: 932ms	remaining: 1.08s
464:	learn: 0.2088423	total: 933ms	remaining: 1.07s
465:	learn: 0.2087110	total: 935ms	remaining: 1.07s
466:	learn: 0.2085945	total: 937ms	remaining: 1.07s
467:	learn: 0.2083919	total: 939ms	remaining: 1.07s
468:	learn: 0.2082374	total: 941ms	remaining: 1.06s
469:	learn: 0.2081489	total: 942ms	remaining: 1.06s
470:	learn: 0.2079327	total: 944ms	remaining: 1.06s
471:	learn: 0.2077617	total: 946ms	remaining: 1.06s
472:	learn: 0.2076093	total: 948ms	remaining: 1.05s
473:	learn: 0.2074996	total: 950ms	remaining: 1.05s
474:	learn: 

627:	learn: 0.1851238	total: 1.26s	remaining: 744ms
628:	learn: 0.1850212	total: 1.26s	remaining: 742ms
629:	learn: 0.1848471	total: 1.26s	remaining: 740ms
630:	learn: 0.1847188	total: 1.26s	remaining: 739ms
631:	learn: 0.1846325	total: 1.26s	remaining: 737ms
632:	learn: 0.1843630	total: 1.27s	remaining: 735ms
633:	learn: 0.1842420	total: 1.27s	remaining: 733ms
634:	learn: 0.1841383	total: 1.27s	remaining: 731ms
635:	learn: 0.1840639	total: 1.27s	remaining: 729ms
636:	learn: 0.1839413	total: 1.27s	remaining: 727ms
637:	learn: 0.1838088	total: 1.28s	remaining: 725ms
638:	learn: 0.1837133	total: 1.28s	remaining: 723ms
639:	learn: 0.1835152	total: 1.28s	remaining: 721ms
640:	learn: 0.1832698	total: 1.28s	remaining: 719ms
641:	learn: 0.1831933	total: 1.29s	remaining: 717ms
642:	learn: 0.1830861	total: 1.29s	remaining: 715ms
643:	learn: 0.1829897	total: 1.29s	remaining: 713ms
644:	learn: 0.1829034	total: 1.29s	remaining: 711ms
645:	learn: 0.1827147	total: 1.29s	remaining: 709ms
646:	learn: 

787:	learn: 0.1673669	total: 1.59s	remaining: 428ms
788:	learn: 0.1670906	total: 1.59s	remaining: 426ms
789:	learn: 0.1669234	total: 1.59s	remaining: 424ms
790:	learn: 0.1668150	total: 1.6s	remaining: 422ms
791:	learn: 0.1667121	total: 1.6s	remaining: 420ms
792:	learn: 0.1666220	total: 1.6s	remaining: 418ms
793:	learn: 0.1664968	total: 1.6s	remaining: 416ms
794:	learn: 0.1664003	total: 1.6s	remaining: 414ms
795:	learn: 0.1662977	total: 1.6s	remaining: 411ms
796:	learn: 0.1662007	total: 1.61s	remaining: 409ms
797:	learn: 0.1660864	total: 1.61s	remaining: 407ms
798:	learn: 0.1659176	total: 1.61s	remaining: 405ms
799:	learn: 0.1657910	total: 1.61s	remaining: 403ms
800:	learn: 0.1656721	total: 1.61s	remaining: 401ms
801:	learn: 0.1655409	total: 1.62s	remaining: 399ms
802:	learn: 0.1654505	total: 1.62s	remaining: 397ms
803:	learn: 0.1652387	total: 1.62s	remaining: 395ms
804:	learn: 0.1651942	total: 1.62s	remaining: 393ms
805:	learn: 0.1650774	total: 1.62s	remaining: 391ms
806:	learn: 0.1649

950:	learn: 0.1519884	total: 1.92s	remaining: 99ms
951:	learn: 0.1519322	total: 1.92s	remaining: 97ms
952:	learn: 0.1518240	total: 1.93s	remaining: 95ms
953:	learn: 0.1517134	total: 1.93s	remaining: 92.9ms
954:	learn: 0.1516065	total: 1.93s	remaining: 90.9ms
955:	learn: 0.1515601	total: 1.93s	remaining: 88.9ms
956:	learn: 0.1514955	total: 1.93s	remaining: 86.9ms
957:	learn: 0.1514210	total: 1.94s	remaining: 84.9ms
958:	learn: 0.1513353	total: 1.94s	remaining: 82.8ms
959:	learn: 0.1512570	total: 1.94s	remaining: 80.8ms
960:	learn: 0.1511640	total: 1.94s	remaining: 78.8ms
961:	learn: 0.1509602	total: 1.94s	remaining: 76.8ms
962:	learn: 0.1508643	total: 1.95s	remaining: 74.7ms
963:	learn: 0.1507236	total: 1.95s	remaining: 72.7ms
964:	learn: 0.1506586	total: 1.95s	remaining: 70.7ms
965:	learn: 0.1505713	total: 1.95s	remaining: 68.7ms
966:	learn: 0.1504742	total: 1.95s	remaining: 66.7ms
967:	learn: 0.1503889	total: 1.96s	remaining: 64.7ms
968:	learn: 0.1503448	total: 1.96s	remaining: 62.6ms

,Customer Id,Cost
0,fffe3400310033003300,270.726868
1,fffe3600350035003400,271.700104
2,fffe3700360030003500,180.514130
3,fffe350038003600,196.538879
4,fffe3500390032003500,316.353851
...,...,...
3495,fffe3200300037003100,1216.052856
3496,fffe390031003100,2714.777832
3497,fffe3200350031003700,605.720459
3498,fffe3100380031003000,272.737396


In [98]:
# from sklearn.model_selection import KFold
# # grid search
# model = xgb.XGBRegressor()

# param_grid = {
#         'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
#         #'min_child_weight': np.arange(0.0001, 0.5, 0.001),
#         #'gamma': np.arange(0.0,40.0,0.005),
#         'learning_rate': np.arange(0.0005,0.3,0.0005),
#         'subsample': np.arange(0.01,1.0,0.01),
#         #'colsample_bylevel': np.round(np.arange(0.1,1.0,0.01)),
#         'colsample_bytree': np.arange(0.1,1.0,0.01)}

# kfold = KFold(n_splits=10, shuffle=True, random_state=10)
# grid_search = RandomizedSearchCV(model, param_grid, n_iter = 100, cv=kfold, n_jobs=-1, verbose=True)

In [99]:
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  2.7min finished


RandomizedSearchCV(cv=KFold(n_splits=10, random_state=10, shuffle=True),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=na...
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0

In [100]:
#scoring(y_test, grid_search.predict(X_test))

99.87858821007251

In [101]:
cat_grid_rs = {'depth': [6,8,10],
               'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.20],
               'iterations': [100, 500, 800, 1000, 2000]}

kfold = KFold(n_splits=10, shuffle=True, random_state=10)
grid_search = RandomizedSearchCV(CatBoostRegressor(), cat_grid_rs, n_iter = 100, cv=kfold, n_jobs=-1, verbose=True)
grid_search.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 75 is smaller than n_iter=100. Running 75 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 24.5min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed: 82.4min finished


0:	learn: 1.5839399	total: 3ms	remaining: 5.99s
1:	learn: 1.5208951	total: 5.18ms	remaining: 5.18s
2:	learn: 1.4647293	total: 7.14ms	remaining: 4.75s
3:	learn: 1.4088076	total: 9.03ms	remaining: 4.5s
4:	learn: 1.3558307	total: 10.9ms	remaining: 4.33s
5:	learn: 1.3038420	total: 12.7ms	remaining: 4.22s
6:	learn: 1.2562220	total: 14.6ms	remaining: 4.16s
7:	learn: 1.2104501	total: 16.5ms	remaining: 4.12s
8:	learn: 1.1665377	total: 18.5ms	remaining: 4.1s
9:	learn: 1.1243074	total: 20.6ms	remaining: 4.09s
10:	learn: 1.0837702	total: 22.4ms	remaining: 4.04s
11:	learn: 1.0454754	total: 24.1ms	remaining: 4s
12:	learn: 1.0099401	total: 25.9ms	remaining: 3.96s
13:	learn: 0.9760347	total: 27.6ms	remaining: 3.92s
14:	learn: 0.9448006	total: 29.6ms	remaining: 3.92s
15:	learn: 0.9144333	total: 31.6ms	remaining: 3.92s
16:	learn: 0.8839126	total: 33.5ms	remaining: 3.9s
17:	learn: 0.8580731	total: 35.4ms	remaining: 3.9s
18:	learn: 0.8321781	total: 37.4ms	remaining: 3.9s
19:	learn: 0.8083227	total: 39.2m

170:	learn: 0.3101681	total: 337ms	remaining: 3.6s
171:	learn: 0.3095754	total: 339ms	remaining: 3.6s
172:	learn: 0.3085050	total: 341ms	remaining: 3.6s
173:	learn: 0.3080460	total: 343ms	remaining: 3.6s
174:	learn: 0.3075345	total: 345ms	remaining: 3.6s
175:	learn: 0.3070503	total: 347ms	remaining: 3.6s
176:	learn: 0.3063924	total: 349ms	remaining: 3.6s
177:	learn: 0.3055103	total: 354ms	remaining: 3.62s
178:	learn: 0.3048320	total: 356ms	remaining: 3.62s
179:	learn: 0.3042840	total: 358ms	remaining: 3.62s
180:	learn: 0.3038888	total: 360ms	remaining: 3.62s
181:	learn: 0.3034599	total: 362ms	remaining: 3.62s
182:	learn: 0.3028942	total: 365ms	remaining: 3.62s
183:	learn: 0.3024162	total: 367ms	remaining: 3.62s
184:	learn: 0.3016979	total: 369ms	remaining: 3.62s
185:	learn: 0.3005950	total: 371ms	remaining: 3.62s
186:	learn: 0.3000758	total: 373ms	remaining: 3.62s
187:	learn: 0.2995080	total: 376ms	remaining: 3.62s
188:	learn: 0.2991887	total: 378ms	remaining: 3.62s
189:	learn: 0.29859

...

1382:	learn: 0.1258069	total: 2.87s	remaining: 1.28s
1383:	learn: 0.1257505	total: 2.87s	remaining: 1.28s
1384:	learn: 0.1257018	total: 2.87s	remaining: 1.27s
1385:	learn: 0.1256671	total: 2.87s	remaining: 1.27s
1386:	learn: 0.1256476	total: 2.87s	remaining: 1.27s
1387:	learn: 0.1256103	total: 2.88s	remaining: 1.27s
1388:	learn: 0.1255759	total: 2.88s	remaining: 1.26s
1389:	learn: 0.1255016	total: 2.88s	remaining: 1.26s
1390:	learn: 0.1253669	total: 2.88s	remaining: 1.26s
1391:	learn: 0.1252386	total: 2.88s	remaining: 1.26s
1392:	learn: 0.1251862	total: 2.89s	remaining: 1.26s
1393:	learn: 0.1250658	total: 2.89s	remaining: 1.25s
1394:	learn: 0.1250025	total: 2.89s	remaining: 1.25s
1395:	learn: 0.1249429	total: 2.89s	remaining: 1.25s
1396:	learn: 0.1248735	total: 2.89s	remaining: 1.25s
1397:	learn: 0.1248102	total: 2.9s	remaining: 1.25s
1398:	learn: 0.1246968	total: 2.9s	remaining: 1.25s
1399:	learn: 0.1245663	total: 2.9s	remaining: 1.24s
1400:	learn: 0.1244949	total: 2.9s	remaining: 1.2

1549:	learn: 0.1161806	total: 3.21s	remaining: 931ms
1550:	learn: 0.1161306	total: 3.21s	remaining: 929ms
1551:	learn: 0.1160903	total: 3.21s	remaining: 927ms
1552:	learn: 0.1160408	total: 3.21s	remaining: 925ms
1553:	learn: 0.1159760	total: 3.22s	remaining: 923ms
1554:	learn: 0.1159537	total: 3.22s	remaining: 921ms
1555:	learn: 0.1159108	total: 3.22s	remaining: 919ms
1556:	learn: 0.1158725	total: 3.22s	remaining: 917ms
1557:	learn: 0.1157838	total: 3.23s	remaining: 915ms
1558:	learn: 0.1157477	total: 3.23s	remaining: 913ms
1559:	learn: 0.1156831	total: 3.23s	remaining: 911ms
1560:	learn: 0.1155914	total: 3.23s	remaining: 909ms
1561:	learn: 0.1155390	total: 3.23s	remaining: 907ms
1562:	learn: 0.1154937	total: 3.23s	remaining: 905ms
1563:	learn: 0.1154539	total: 3.24s	remaining: 903ms
1564:	learn: 0.1153799	total: 3.24s	remaining: 901ms
1565:	learn: 0.1153279	total: 3.24s	remaining: 899ms
1566:	learn: 0.1152564	total: 3.25s	remaining: 897ms
1567:	learn: 0.1152051	total: 3.25s	remaining:

1714:	learn: 0.1082928	total: 3.55s	remaining: 590ms
1715:	learn: 0.1082640	total: 3.55s	remaining: 588ms
1716:	learn: 0.1082290	total: 3.55s	remaining: 586ms
1717:	learn: 0.1081936	total: 3.56s	remaining: 584ms
1718:	learn: 0.1081380	total: 3.56s	remaining: 582ms
1719:	learn: 0.1081084	total: 3.56s	remaining: 580ms
1720:	learn: 0.1080649	total: 3.56s	remaining: 578ms
1721:	learn: 0.1080202	total: 3.56s	remaining: 576ms
1722:	learn: 0.1079813	total: 3.57s	remaining: 573ms
1723:	learn: 0.1079339	total: 3.57s	remaining: 571ms
1724:	learn: 0.1079001	total: 3.57s	remaining: 569ms
1725:	learn: 0.1078343	total: 3.57s	remaining: 567ms
1726:	learn: 0.1077595	total: 3.58s	remaining: 565ms
1727:	learn: 0.1077086	total: 3.58s	remaining: 563ms
1728:	learn: 0.1076654	total: 3.58s	remaining: 561ms
1729:	learn: 0.1076189	total: 3.58s	remaining: 559ms
1730:	learn: 0.1076128	total: 3.58s	remaining: 557ms
1731:	learn: 0.1075279	total: 3.59s	remaining: 555ms
1732:	learn: 0.1074843	total: 3.59s	remaining:

1876:	learn: 0.1012339	total: 3.89s	remaining: 255ms
1877:	learn: 0.1011983	total: 3.89s	remaining: 253ms
1878:	learn: 0.1011844	total: 3.89s	remaining: 251ms
1879:	learn: 0.1011400	total: 3.9s	remaining: 249ms
1880:	learn: 0.1011059	total: 3.9s	remaining: 247ms
1881:	learn: 0.1010615	total: 3.9s	remaining: 245ms
1882:	learn: 0.1010192	total: 3.9s	remaining: 242ms
1883:	learn: 0.1009865	total: 3.9s	remaining: 240ms
1884:	learn: 0.1009045	total: 3.91s	remaining: 238ms
1885:	learn: 0.1008606	total: 3.91s	remaining: 236ms
1886:	learn: 0.1008050	total: 3.91s	remaining: 234ms
1887:	learn: 0.1007662	total: 3.91s	remaining: 232ms
1888:	learn: 0.1006887	total: 3.91s	remaining: 230ms
1889:	learn: 0.1006595	total: 3.92s	remaining: 228ms
1890:	learn: 0.1005879	total: 3.92s	remaining: 226ms
1891:	learn: 0.1005192	total: 3.92s	remaining: 224ms
1892:	learn: 0.1004766	total: 3.92s	remaining: 222ms
1893:	learn: 0.1004134	total: 3.92s	remaining: 220ms
1894:	learn: 0.1003669	total: 3.93s	remaining: 218m

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=10, shuffle=True),
                   estimator=<catboost.core.CatBoostRegressor object at 0x00000184A0CFAD00>,
                   n_iter=100, n_jobs=-1,
                   param_distributions={'depth': [6, 8, 10],
                                        'iterations': [100, 500, 800, 1000,
                                                       2000],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.15,
                                                          0.2]},
                   verbose=True)

In [102]:
scoring(y_test, grid_search.predict(X_test))

99.8963061214804

In [103]:
grid_search.best_estimator_.fit(X, y)

0:	learn: 1.5834983	total: 3.93ms	remaining: 7.86s
1:	learn: 1.5201743	total: 6.33ms	remaining: 6.32s
2:	learn: 1.4636891	total: 8.71ms	remaining: 5.8s
3:	learn: 1.4056077	total: 10.8ms	remaining: 5.41s
4:	learn: 1.3493478	total: 12.9ms	remaining: 5.14s
5:	learn: 1.2972329	total: 15ms	remaining: 4.98s
6:	learn: 1.2477841	total: 17ms	remaining: 4.83s
7:	learn: 1.2017740	total: 19.1ms	remaining: 4.75s
8:	learn: 1.1570057	total: 21.1ms	remaining: 4.67s
9:	learn: 1.1148547	total: 23.1ms	remaining: 4.59s
10:	learn: 1.0744484	total: 25ms	remaining: 4.52s
11:	learn: 1.0376432	total: 27ms	remaining: 4.47s
12:	learn: 1.0026523	total: 29.4ms	remaining: 4.5s
13:	learn: 0.9699323	total: 32.1ms	remaining: 4.56s
14:	learn: 0.9390559	total: 34.5ms	remaining: 4.57s
15:	learn: 0.9084478	total: 36.5ms	remaining: 4.53s
16:	learn: 0.8793064	total: 38.7ms	remaining: 4.51s
17:	learn: 0.8533772	total: 42ms	remaining: 4.62s
18:	learn: 0.8279950	total: 45.1ms	remaining: 4.7s
19:	learn: 0.8046585	total: 47.6ms	

170:	learn: 0.3168245	total: 348ms	remaining: 3.72s
171:	learn: 0.3161962	total: 350ms	remaining: 3.72s
172:	learn: 0.3158638	total: 352ms	remaining: 3.72s
173:	learn: 0.3154907	total: 354ms	remaining: 3.72s
174:	learn: 0.3151309	total: 356ms	remaining: 3.71s
175:	learn: 0.3147808	total: 358ms	remaining: 3.71s
176:	learn: 0.3136500	total: 360ms	remaining: 3.71s
177:	learn: 0.3133769	total: 363ms	remaining: 3.71s
178:	learn: 0.3128444	total: 365ms	remaining: 3.71s
179:	learn: 0.3123534	total: 367ms	remaining: 3.71s
180:	learn: 0.3112878	total: 369ms	remaining: 3.71s
181:	learn: 0.3107300	total: 371ms	remaining: 3.71s
182:	learn: 0.3102606	total: 373ms	remaining: 3.71s
183:	learn: 0.3098947	total: 376ms	remaining: 3.71s
184:	learn: 0.3091791	total: 378ms	remaining: 3.71s
185:	learn: 0.3084153	total: 380ms	remaining: 3.71s
186:	learn: 0.3078858	total: 383ms	remaining: 3.71s
187:	learn: 0.3074845	total: 385ms	remaining: 3.71s
188:	learn: 0.3068629	total: 387ms	remaining: 3.71s
189:	learn: 

343:	learn: 0.2460974	total: 687ms	remaining: 3.31s
344:	learn: 0.2457100	total: 689ms	remaining: 3.31s
345:	learn: 0.2455418	total: 691ms	remaining: 3.31s
346:	learn: 0.2453556	total: 694ms	remaining: 3.31s
347:	learn: 0.2451382	total: 696ms	remaining: 3.3s
348:	learn: 0.2449158	total: 698ms	remaining: 3.3s
349:	learn: 0.2447851	total: 700ms	remaining: 3.3s
350:	learn: 0.2445576	total: 702ms	remaining: 3.3s
351:	learn: 0.2441920	total: 704ms	remaining: 3.29s
352:	learn: 0.2439975	total: 706ms	remaining: 3.29s
353:	learn: 0.2436886	total: 708ms	remaining: 3.29s
354:	learn: 0.2435668	total: 710ms	remaining: 3.29s
355:	learn: 0.2433171	total: 712ms	remaining: 3.29s
356:	learn: 0.2431198	total: 714ms	remaining: 3.28s
357:	learn: 0.2429542	total: 715ms	remaining: 3.28s
358:	learn: 0.2425958	total: 717ms	remaining: 3.28s
359:	learn: 0.2424352	total: 719ms	remaining: 3.28s
360:	learn: 0.2422159	total: 721ms	remaining: 3.27s
361:	learn: 0.2421101	total: 724ms	remaining: 3.27s
362:	learn: 0.24

514:	learn: 0.2147152	total: 1.03s	remaining: 2.97s
515:	learn: 0.2146133	total: 1.03s	remaining: 2.97s
516:	learn: 0.2144926	total: 1.03s	remaining: 2.96s
517:	learn: 0.2144121	total: 1.03s	remaining: 2.96s
518:	learn: 0.2142875	total: 1.04s	remaining: 2.96s
519:	learn: 0.2141872	total: 1.04s	remaining: 2.96s
520:	learn: 0.2141016	total: 1.04s	remaining: 2.96s
521:	learn: 0.2139031	total: 1.04s	remaining: 2.96s
522:	learn: 0.2137305	total: 1.05s	remaining: 2.95s
523:	learn: 0.2136396	total: 1.05s	remaining: 2.95s
524:	learn: 0.2135021	total: 1.05s	remaining: 2.95s
525:	learn: 0.2133258	total: 1.05s	remaining: 2.95s
526:	learn: 0.2131941	total: 1.05s	remaining: 2.94s
527:	learn: 0.2130260	total: 1.05s	remaining: 2.94s
528:	learn: 0.2128963	total: 1.06s	remaining: 2.94s
529:	learn: 0.2127312	total: 1.06s	remaining: 2.94s
530:	learn: 0.2126335	total: 1.06s	remaining: 2.94s
531:	learn: 0.2122933	total: 1.06s	remaining: 2.93s
532:	learn: 0.2121434	total: 1.06s	remaining: 2.93s
533:	learn: 

683:	learn: 0.1943749	total: 1.37s	remaining: 2.63s
684:	learn: 0.1942912	total: 1.37s	remaining: 2.63s
685:	learn: 0.1942005	total: 1.37s	remaining: 2.63s
686:	learn: 0.1940529	total: 1.38s	remaining: 2.63s
687:	learn: 0.1939035	total: 1.38s	remaining: 2.63s
688:	learn: 0.1938281	total: 1.38s	remaining: 2.63s
689:	learn: 0.1937075	total: 1.38s	remaining: 2.63s
690:	learn: 0.1936586	total: 1.39s	remaining: 2.63s
691:	learn: 0.1935502	total: 1.39s	remaining: 2.63s
692:	learn: 0.1934561	total: 1.39s	remaining: 2.63s
693:	learn: 0.1933849	total: 1.39s	remaining: 2.62s
694:	learn: 0.1932223	total: 1.4s	remaining: 2.62s
695:	learn: 0.1931107	total: 1.4s	remaining: 2.62s
696:	learn: 0.1930255	total: 1.4s	remaining: 2.62s
697:	learn: 0.1929406	total: 1.4s	remaining: 2.62s
698:	learn: 0.1928177	total: 1.41s	remaining: 2.62s
699:	learn: 0.1927442	total: 1.41s	remaining: 2.61s
700:	learn: 0.1926412	total: 1.41s	remaining: 2.61s
701:	learn: 0.1924733	total: 1.41s	remaining: 2.61s
702:	learn: 0.19

...

1130:	learn: 0.1564055	total: 2.38s	remaining: 1.82s
1131:	learn: 0.1563604	total: 2.38s	remaining: 1.82s
1132:	learn: 0.1563037	total: 2.38s	remaining: 1.82s
1133:	learn: 0.1562480	total: 2.38s	remaining: 1.82s
1134:	learn: 0.1561977	total: 2.38s	remaining: 1.82s
1135:	learn: 0.1561323	total: 2.39s	remaining: 1.82s
1136:	learn: 0.1560639	total: 2.39s	remaining: 1.81s
1137:	learn: 0.1559670	total: 2.39s	remaining: 1.81s
1138:	learn: 0.1559465	total: 2.4s	remaining: 1.81s
1139:	learn: 0.1558632	total: 2.4s	remaining: 1.81s
1140:	learn: 0.1558107	total: 2.4s	remaining: 1.81s
1141:	learn: 0.1557660	total: 2.4s	remaining: 1.81s
1142:	learn: 0.1557105	total: 2.41s	remaining: 1.8s
1143:	learn: 0.1556346	total: 2.41s	remaining: 1.8s
1144:	learn: 0.1556004	total: 2.41s	remaining: 1.8s
1145:	learn: 0.1555224	total: 2.41s	remaining: 1.8s
1146:	learn: 0.1554274	total: 2.42s	remaining: 1.8s
1147:	learn: 0.1553212	total: 2.42s	remaining: 1.79s
1148:	learn: 0.1552512	total: 2.42s	remaining: 1.79s
11

1359:	learn: 0.1422922	total: 2.88s	remaining: 1.36s
1360:	learn: 0.1421824	total: 2.89s	remaining: 1.35s
1361:	learn: 0.1421472	total: 2.89s	remaining: 1.35s
1362:	learn: 0.1420891	total: 2.89s	remaining: 1.35s
1363:	learn: 0.1420403	total: 2.89s	remaining: 1.35s
1364:	learn: 0.1419875	total: 2.9s	remaining: 1.35s
1365:	learn: 0.1419079	total: 2.9s	remaining: 1.34s
1366:	learn: 0.1418761	total: 2.9s	remaining: 1.34s
1367:	learn: 0.1418411	total: 2.9s	remaining: 1.34s
1368:	learn: 0.1417793	total: 2.9s	remaining: 1.34s
1369:	learn: 0.1417335	total: 2.91s	remaining: 1.34s
1370:	learn: 0.1416804	total: 2.91s	remaining: 1.33s
1371:	learn: 0.1415759	total: 2.91s	remaining: 1.33s
1372:	learn: 0.1415085	total: 2.91s	remaining: 1.33s
1373:	learn: 0.1414458	total: 2.91s	remaining: 1.33s
1374:	learn: 0.1413975	total: 2.92s	remaining: 1.32s
1375:	learn: 0.1413613	total: 2.92s	remaining: 1.32s
1376:	learn: 0.1412707	total: 2.92s	remaining: 1.32s
1377:	learn: 0.1412302	total: 2.92s	remaining: 1.32

1530:	learn: 0.1326814	total: 3.23s	remaining: 989ms
1531:	learn: 0.1326629	total: 3.23s	remaining: 986ms
1532:	learn: 0.1326395	total: 3.23s	remaining: 984ms
1533:	learn: 0.1325993	total: 3.23s	remaining: 982ms
1534:	learn: 0.1325405	total: 3.24s	remaining: 981ms
1535:	learn: 0.1324914	total: 3.24s	remaining: 979ms
1536:	learn: 0.1324384	total: 3.24s	remaining: 977ms
1537:	learn: 0.1323905	total: 3.24s	remaining: 974ms
1538:	learn: 0.1323301	total: 3.25s	remaining: 972ms
1539:	learn: 0.1322899	total: 3.25s	remaining: 970ms
1540:	learn: 0.1322484	total: 3.25s	remaining: 968ms
1541:	learn: 0.1321943	total: 3.25s	remaining: 966ms
1542:	learn: 0.1321365	total: 3.25s	remaining: 964ms
1543:	learn: 0.1320959	total: 3.25s	remaining: 962ms
1544:	learn: 0.1320572	total: 3.26s	remaining: 959ms
1545:	learn: 0.1319725	total: 3.26s	remaining: 957ms
1546:	learn: 0.1319263	total: 3.26s	remaining: 955ms
1547:	learn: 0.1318652	total: 3.26s	remaining: 953ms
1548:	learn: 0.1318162	total: 3.27s	remaining:

1702:	learn: 0.1241759	total: 3.57s	remaining: 622ms
1703:	learn: 0.1241296	total: 3.57s	remaining: 620ms
1704:	learn: 0.1240826	total: 3.57s	remaining: 618ms
1705:	learn: 0.1240598	total: 3.57s	remaining: 616ms
1706:	learn: 0.1240117	total: 3.58s	remaining: 614ms
1707:	learn: 0.1239655	total: 3.58s	remaining: 612ms
1708:	learn: 0.1239165	total: 3.58s	remaining: 610ms
1709:	learn: 0.1238281	total: 3.58s	remaining: 608ms
1710:	learn: 0.1237878	total: 3.58s	remaining: 605ms
1711:	learn: 0.1237417	total: 3.59s	remaining: 603ms
1712:	learn: 0.1237112	total: 3.59s	remaining: 601ms
1713:	learn: 0.1236703	total: 3.59s	remaining: 599ms
1714:	learn: 0.1236328	total: 3.59s	remaining: 597ms
1715:	learn: 0.1235712	total: 3.6s	remaining: 595ms
1716:	learn: 0.1235437	total: 3.6s	remaining: 593ms
1717:	learn: 0.1235130	total: 3.6s	remaining: 591ms
1718:	learn: 0.1234482	total: 3.6s	remaining: 589ms
1719:	learn: 0.1233962	total: 3.6s	remaining: 587ms
1720:	learn: 0.1232944	total: 3.6s	remaining: 584ms

1876:	learn: 0.1165897	total: 3.91s	remaining: 256ms
1877:	learn: 0.1164923	total: 3.91s	remaining: 254ms
1878:	learn: 0.1164478	total: 3.91s	remaining: 252ms
1879:	learn: 0.1164054	total: 3.92s	remaining: 250ms
1880:	learn: 0.1163632	total: 3.92s	remaining: 248ms
1881:	learn: 0.1163252	total: 3.92s	remaining: 246ms
1882:	learn: 0.1162859	total: 3.92s	remaining: 244ms
1883:	learn: 0.1162539	total: 3.92s	remaining: 242ms
1884:	learn: 0.1162326	total: 3.93s	remaining: 240ms
1885:	learn: 0.1161971	total: 3.93s	remaining: 238ms
1886:	learn: 0.1161336	total: 3.93s	remaining: 235ms
1887:	learn: 0.1160997	total: 3.94s	remaining: 234ms
1888:	learn: 0.1160497	total: 3.94s	remaining: 232ms
1889:	learn: 0.1160039	total: 3.94s	remaining: 230ms
1890:	learn: 0.1159692	total: 3.95s	remaining: 228ms
1891:	learn: 0.1159352	total: 3.95s	remaining: 225ms
1892:	learn: 0.1158980	total: 3.95s	remaining: 223ms
1893:	learn: 0.1158785	total: 3.95s	remaining: 221ms
1894:	learn: 0.1158254	total: 3.96s	remaining: